# Klassifikation mit Support Vector Machines

In diesem Teil wirst du kennenlernen, wie du mithilfe einer sog. [Support Vector Machine](https://de.wikipedia.org/wiki/Support_Vector_Machine) (SVM) Daten sehr einfach klassifizieren kannst. Die SVM ist nur eine Möglichkeit der Klassifikation, es gibt auch noch weiter Modelle, die du später kennenlernen wirst.

## Daten einladen

Wie gewohnt lädst du die linguistisch analysierten Daten ein:

In [ ]:
import sys, os
ON_COLAB = 'google.colab' in sys.modules

if ON_COLAB:
    os.system("test -f heise-articles-2020.db || wget  https://datanizing.com/heiseacademy/nlp-course/blob/main/99_Common/heise-articles-2020.db.gz && gunzip heise-articles-2020.db.gz")
    newsticker_db = 'heise-articles-2020.db'
else:
    newsticker_db = '../99_Common/heise-articles-2020.db'

In [ ]:
import sqlite3 
import pandas as pd

sql = sqlite3.connect(newsticker_db)
df = pd.read_sql("SELECT * FROM nlp_articles WHERE datePublished<'2021-01-01' ORDER BY datePublished", 
                 sql, index_col="id", parse_dates=["datePublished"])

Alle diese Felder kommen damit prinzipiell als Kategorien in Frage. Betrachte sie nun nacheinander:

## Kandidat: `author`

Du benögtigst mindestens ein paar Hundert Datensätze, um den Klassifikator zu trainieren. Wenn du für Artikel die Autoren vorhersagen möchtest, müsstest du daher Autoren finden, die mindestens 100 Artikel geschrieben haben. Betrachte dazu die Top-20-Autoren:

In [ ]:
top_authors = df.groupby("author").count().sort_values("title", ascending=False).head(20)[["title"]]

Damit du zu einer ausgeglichenen Trainingsmenge kommst, suchst du den Top-Autor mit den wenigsten Artikeln:

In [ ]:
min_articles = min(top_authors["title"])

Nun konstruierst du einen `DataFrame`, der von allen Autoren gleich viele Artikel enthält:

In [ ]:
adf = pd.concat([df[df["author"] == author].sample(min_articles, random_state=42)
                     for author in top_authors.index.values])

Nun kannst du die Daten vektorisieren:

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from spacy.lang.de.stop_words import STOP_WORDS as stop_words
tfidf_vectorizer = TfidfVectorizer(stop_words=stop_words, min_df=2)
tfidf_vectors = tfidf_vectorizer.fit_transform(adf["nav"])
tfidf_vectors

Das Modell lässt sich sehr schnell trainieren:

In [ ]:
from sklearn.svm import SVC
svc = SVC()
svc.fit(tfidf_vectors, adf["author"])

Eine Vorhersage kannst du ebenso einfach durchführen und die Ergebnisse im `DataFrame` abspeichern:

In [ ]:
adf["predicted_author"] = svc.predict(tfidf_vectors)

Schließlich kannst du zählen, wie häufig das Modell richtig und falsch klassifiziert hat:

In [ ]:
print(len(adf[adf["author"] == adf["predicted_author"]]))
print(len(adf[adf["author"] != adf["predicted_author"]]))

Das hat *außerordentlich* gut funktioniert. Das Modell ist so gut trainiert, dass es nur in 13 von 2.600 Fällen eine falsche Vorhersage liefert, das sind nur 0,5% falsche Werte. Allerdings könnte es sein, dass es die Daten nur auswendig gelernt hat. Das werden wir später untersuchen!

## Kandidat: `keywords`

Auch die Keywords könnten eine gute Wahl für eine bereits vorklassifizierte Kategorie sein. Allerdings sind pro Meldung mehrere Keywords vergeben. Wenn du die zählen möchtest, machst du das am besten mit einem `Counter`:

In [ ]:
from collections import Counter
keywords = Counter([keyword for keywords in df["keywords"] for keyword in str(keywords).split(", ")])
top_keywords = [keyword[0] for keyword in keywords.most_common(20)]

Nun musst du allerdings anders vorgehen. Für jedes Keyword trainierst du ein eigenes Modell, indem du jeweils einen `DataFrame` konstruierst, der in gleicher Zahl positive und negative Beispiele für das Keywords enthält:

In [ ]:
for keyword in top_keywords:
    # DataFrame mit/ohne Keyword bestimmen
    k_pos = df[df["keywords"].map(str).str.contains(keyword)].copy()
    k_pos["keyword"] = 1
    k_neg = df[~ df["keywords"].map(str).str.contains(keyword)].copy()
    k_neg["keyword"] = 0
    
    # kleinste Länge ausrechnen
    min_keyword = min(len(k_pos), len(k_neg))
    kdf = pd.concat([k_pos.sample(min_keyword, random_state=42),
                     k_neg.sample(min_keyword, random_state=42)])
    
    # vektorisiere
    tfidf_vectorizer = TfidfVectorizer(stop_words=stop_words, min_df=2)
    tfidf_vectors = tfidf_vectorizer.fit_transform(kdf["nav"])
    
    # trainieren
    svc = SVC()
    svc.fit(tfidf_vectors, kdf["keyword"])
    
    # vorhersagen
    kdf["predicted_keyword"] = svc.predict(tfidf_vectors)
    
    # richtig/falsch berechnen
    print(keyword)
    print(len(kdf[kdf["keyword"] == kdf["predicted_keyword"]]))
    print(len(kdf[kdf["keyword"] != kdf["predicted_keyword"]]))

Auch diese Klassifikation hat sehr gut funktioniert!

## Kandidat: `commentCount`

Zunächst normalisierst du die Kommentare:

In [ ]:
df["normalizedCommentCount"] = df["commentCount"].fillna(0).map(int)
df.loc[df["normalizedCommentCount"]>500, "normalizedCommentCount"] = 500

Dann konstruierst du zwei `DataFrame`, in denen erfolgreich und nicht erfolgreiche Posts enthalten sind:

In [ ]:
df_success = df[df["normalizedCommentCount"]>50].copy()
df_success["success"] = 1

df_no_success = df[df["normalizedCommentCount"]<10].copy()
df_no_success["success"] = 0

Du berechnest die Größe des kleineren `DataFrame`:

In [ ]:
min_success = min(len(df_success), len(df_no_success))

Und erzeugst ein ausgeglichenes Trainingsset:

In [ ]:
sdf = pd.concat([df_success.sample(min_success, random_state=42),
                 df_no_success.sample(min_success, random_state=42)])

Die Daten vektorisierst du nun:

In [ ]:
tfidf_vectorizer = TfidfVectorizer(stop_words=stop_words, min_df=2)
tfidf_vectors = tfidf_vectorizer.fit_transform(sdf["nav"])
tfidf_vectors

Trainierst das Modell:

In [ ]:
svc = SVC()
svc.fit(tfidf_vectors, sdf["success"])

Du führst nun die Vorhersage durch:

In [ ]:
sdf["predicted_success"] = svc.predict(tfidf_vectors)

Und bewertest das Ergebnis:

In [ ]:
print(len(sdf[sdf["success"] == sdf["predicted_success"]]))
print(len(sdf[sdf["success"] != sdf["predicted_success"]]))

Auch das hat extrem gut funktioniert. Es wäre super, wenn man aus dem Artikel schon vorher prädizieren könnte, wie erfolgreich der Artikel wird. Die Heise-Redakteure können das wahrscheinlich intuitiv!

Wir werden später allerdings überprüfen müssen, ob der Klassifikator die Daten nicht nur auswendig gelernt hat! 

## Klassifikation ist nicht schwierig

Wie du gesehen hast, kann man einen Klassifikator schnell trainieren und in dem Datenset funktionieren auch alle Klassifikatoren wirklich sehr gut.

Später überprüfst du, wie gut der Algorithmus mit unbekannten Daten umgehen (also *abstrahieren*) kann und du wirst Gütekriterien kennenlernen.